In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
uploaded1 = files.upload() # evaluation script


Mounted at /content/drive


Saving evaluation.py to evaluation.py


Install Dependicies


In [ ]:
!pip install transformers
!pip install datasets
!pip install rouge-score
!pip install sentencepiece
!pip install sacrebleu==1.5.1
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.8 MB/s eta 0:00:0

Import Libraries

In [ ]:
import pandas as pd
import datasets
from datasets import Metric, load_dataset, load_metric, Dataset

import torch
from torch import optim
from torch.utils.data import DataLoader

import transformers
from transformers import (
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    PreTrainedTokenizer
)

In [ ]:
torch.cuda.empty_cache()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

###PREPARE DATA: PART 1
read the data into DataFrame; split into train, test, val; convert to hugging face Datasets


In [ ]:
# # read the data
# df = pd.read_csv('/content/drive/MyDrive/coNaLa-data/conala_alltrain.csv')

# # get number of sampels we wish to use
# df = df.iloc[0:50000]

# # Split dataframe into train, validation
# train_df = df.sample(frac=0.9,random_state=29)
# val_df = df.drop(train_df.index)
# train_df.reset_index(drop=True, inplace=True)
# val_df.reset_index(drop=True, inplace=True)

# # convert dataframes into huggingface Datasets
# train_dataset = Dataset.from_pandas(train_df)
# val_dataset = Dataset.from_pandas(val_df)

In [ ]:
# import the highe quality data
hq_train_df = pd.read_csv('/content/drive/MyDrive/coNaLa-data/hq_train.csv')
hq_test_df = pd.read_csv('/content/drive/MyDrive/coNaLa-data/hq_test.csv')
hq_val_df = pd.read_csv('/content/drive/MyDrive/coNaLa-data/hq_val.csv')

# import the mined data
mined_df = pd.read_csv('/content/drive/MyDrive/coNaLa-data/conala_train_clean.csv')

In [ ]:
train_df = pd.concat([hq_train_df, mined_df])
train_df = train_df.iloc[0:35000]
train_df.reset_index(inplace=True)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(hq_val_df)

###TOKENIZER

In [ ]:
# get the tokenization method used in the Marian model
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-nl", use_fast = True)

###MODEL
get the pretrained Marian model

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-nl")

###PREPARE THE DATA: PART 2

In [ ]:
# input length and output length; change maybe?
encoder_length = 32
decoder_length = 32
batch_size = 1

# map data correctly
def map_to_encoder_decoder_inputs(batch):

    # assign tokens to the input; if  input has to many tokens truncate it; if input has too few tokens pad it; creates input id and attention mask for each token
    inputs = tokenizer(batch["intent"], padding="max_length", truncation=True, max_length=encoder_length)  
    # do the same for output
    outputs = tokenizer(batch["snippet"], padding="max_length", truncation=True, max_length=decoder_length)  

    # record information of the batch
    batch["input_ids"] = inputs.input_ids                              # ids of the input tokens
    batch["attention_mask"] = inputs.attention_mask                    # attention mask of the input tokens
    batch["decoder_input_ids"] = outputs.input_ids                     # ids of the output (snippet) tokens
    batch["labels"] = outputs.input_ids.copy()                         # a copy of the output ids
    batch["decoder_attention_mask"] = outputs.attention_mask           # attention mask of the snippet tokens
    
    # make checks that all batch input_ids and output_ids are the specified encoder and decoder length
    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])
    
    return batch

In [ ]:
# make train dataset ready; apply the map_to_encoder_decoder_inputs; remove intent and snippet columns since we have their token representation
train_data = train_dataset.map( map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['intent', 'snippet'])
train_data.set_format(type="torch",  columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"])

# do the same for the validation dataset
val_data = val_dataset.map( map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['intent', 'snippet'])
val_data.set_format(type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"])

# create an object that will form batches
my_data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

###TRAINING

In [ ]:
# import our evaluation metrics
import evaluation
evaluator = evaluation.CodeGenerationEvaluator(tokenizer, device, smooth_bleu=True)

# set training arguments (expermiment with?)
training_args = Seq2SeqTrainingArguments(
    output_dir="./Marian-Training",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    num_train_epochs=10,
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 1e-5,
    weight_decay=0.01, 
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
)
 

# setup the trainer
trainer = Seq2SeqTrainer(
    model = model,
    tokenizer = tokenizer,
    args = training_args,
    compute_metrics=evaluator,
    data_collator = my_data_collator,
    train_dataset = train_data,
    eval_dataset = val_data,
)

/content/evaluation.py:32: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.sacre_bleu: Metric = load_metric('sacrebleu')


In [ ]:
# train the model; then save it
trainer.train()

trainer.save_model('/content/drive/MyDrive/fine_tuned_marian_hq_plue_mined')
trainer.save_state()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,1.577500,1.310409,16.616872,15.234002,41.704945,20.411724,11.507077,12.870548,25.338029,38.261211,17.637294,10.875306
2,1.251300,1.053849,21.581889,20.046149,48.267483,26.213962,16.442171,16.230946,29.242969,44.909785,23.236618,15.816136
3,1.089400,0.916664,26.351609,24.897150,50.912576,29.903315,20.416931,20.628528,33.841057,47.837567,27.343335,19.833778
4,1.036900,0.832366,30.418514,28.837093,54.465000,34.434562,24.965021,24.302037,37.533285,51.529708,31.774313,24.383172
5,0.908600,0.774204,33.346885,31.755703,56.642555,36.841614,27.586890,27.069611,40.132950,53.708829,34.235116,26.926682
6,0.844500,0.728921,35.848876,34.474707,58.138404,39.110300,30.306340,29.703520,42.431247,55.366661,36.693136,29.600829
7,0.824800,0.697043,38.223846,36.819723,60.548893,42.095501,33.382182,32.531699,44.939595,57.915456,39.744291,32.782019
8,0.756000,0.675466,40.132595,38.778515,60.808436,42.930818,34.526274,33.817979,46.032348,58.255138,40.616206,33.990605
9,0.755600,0.660518,40.965144,39.642503,61.031967,43.223023,34.875912,35.003809,46.908202,58.471551,40.986118,34.264489
10,0.737400,0.656836,41.547070,40.213407,62.111774,44.513506,36.263082,35.356423,47.331154,59.641937,42.281977,35.675704


TEST THE TRAINED MODEL ON TEST DATA

In [ ]:
# load the trained model
trained_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/fine_tuned_marian_hq_plue_mined")
trained_model.to("cuda")

new_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/fine_tuned_marian_hq_plue_mined", use_fast=True)

In [ ]:
test_dataset = Dataset.from_pandas(hq_test_df)

In [ ]:
# evaluate the trained model
evaluator = evaluation.CodeGenerationEvaluator(new_tokenizer, device,smooth_bleu=True)

# define a function that can do code generation on a batch
def generate_new_code(batch):
    inputs = new_tokenizer(batch["intent"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = trained_model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = new_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_code"] = output_str

    return batch

# generate code for test natural language
results = test_dataset.map(generate_new_code, batched=True, batch_size=batch_size)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
# get the bleu score of the results
bleu_score = 0

for i in range(len(results)):

  ref = results['snippet'][i]
  pred = results['pred_code'][i]

  if pred is not None and pred != "":

      if ref is not None and ref != "":
       
        metrics = evaluator.evaluate([pred], [ref])

        bleu_score += metrics['BLEU']

In [ ]:
print(bleu_score/len(hq_test_df))

33.43752674771536


HAVE FUN WITH THE MODEL

In [ ]:
# define a function that takes in natural language and returns code
def generate_code(NL):
    inputs = new_tokenizer([NL], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = trained_model.generate(input_ids, attention_mask=attention_mask)

    # decode the outputs and remove any special tokens
    output_code = new_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return output_code[0]

In [ ]:
print(generate_code('create an array of random numbers'))

▁random_array([random.random() for _ in range(10)])
